# Chappeau II: Nested JSON is My Passion

My Custom Model Items data+resource pack has the chance now to be datapack-free thanks to 1.21.5 including the ability to apply custom models and textures based on item components (read: **custom names**).

This is hugely exciting... but it also means a giant refactor.

In [1]:
import json
from pathlib import Path

from IPython.display import Markdown, display

## Caffeination

While I do think I'll preserve the coffee-brewing datapack, I also want this functionality to be available via resource pack alone. So what I'm going to do is make it so the custom models are applied to generically-named items with the appropriate potion strength, _or_ the _specifically-named_ items with the matching contents and strength.

In [2]:
coffees: list[tuple[str, str, tuple[str, ...]]] = [
    # (effect, coffee, variants)
    ("night_vision", "Auromar Geisha", ("long",)),
    ("leaping", "Legender Especial", ("long", "strong")),
    ("fire_resistance", "La Hermosa", ("long",)),
    ("swiftness", "Yirgacheffe", ("long", "strong")),
    ("water_breathing", "Kona", ("long",)),
    ("healing", "Ndumberi Peaberry", ("strong",)),
    ("regeneration", "Cimitrio Luna", ("long", "strong")),
    ("strength", "Minga Cabrera", ("long", "strong")),
    ("slow_falling", "Xejuyu", ("long",)),
]

So for the outer selector, we're going to use `minecraft:potion_contents` (meaning we'll do names for the inner part)

In [3]:
potion_spec = {
    "model": {
        "type": "minecraft:select",
        "property": "component",
        "component": "minecraft:potion_contents",
        "fallback": {"type": "model", "model": "minecraft:item/potion"},
        "cases": [],
    }
}
for effect, coffee, variants in coffees:
    for variant in ("", *variants):
        if variant:
            potion_content = f"minecraft:{variant}_{effect}"
        else:
            potion_content = f"minecraft:{effect}"
        outer_case = {
            "when": potion_content,
            "model": {
                "type": "minecraft:select",
                "property": "component",
                "component": "custom_name",
                "fallback": {"type": "model", "model": "item/potion"},
                "cases": [],
            },
        }
        if variant == "":
            qualifier = "12oz"
            model = "caffeinate:item/coffee_small"
        elif variant == "long":
            qualifier = "16oz"
            model = "caffeinate:item/coffee_large"
        else:  # if qualifier == "strong"
            qualifier = "Espresso"
            model = "caffeinate:item/espresso"
        valid_names = ["Coffee", coffee, f"{coffee} ({qualifier})"]
        if variant == "strong":
            valid_names.append("Espresso")
        if variant == "":
            valid_names.extend(["Coffee (12oz)", "Coffee (Small)"])
        if variant == "long":
            valid_names.extend(["Coffee (16oz)", "Coffee (Large)"])
        for name in valid_names:
            outer_case["model"]["cases"].append(  # type: ignore[index]
                {"when": name, "model": {"type": "model", "model": model}}
            )
        potion_spec["model"]["cases"].append(outer_case)  # type: ignore[attr-defined]

print(json.dumps(potion_spec, indent=2, sort_keys=False))

{
  "model": {
    "type": "minecraft:select",
    "property": "component",
    "component": "minecraft:potion_contents",
    "fallback": {
      "type": "model",
      "model": "minecraft:item/potion"
    },
    "cases": [
      {
        "when": "minecraft:night_vision",
        "model": {
          "type": "minecraft:select",
          "property": "component",
          "component": "custom_name",
          "fallback": {
            "type": "model",
            "model": "item/potion"
          },
          "cases": [
            {
              "when": "Coffee",
              "model": {
                "type": "model",
                "model": "caffeinate:item/coffee_small"
              }
            },
            {
              "when": "Auromar Geisha",
              "model": {
                "type": "model",
                "model": "caffeinate:item/coffee_small"
              }
            },
            {
              "when": "Auromar Geisha (12oz)",
              "model": 

### Datapack

So from now on, all the datapack needs to do is apply the name (doesn't need to set the custom model data value).

In [4]:
template = """
execute if block ~ ~ ~
minecraft:brewing_stand{Items:[{components:{"minecraft:potion_contents":{potion:"minecraft:MODIFIER_EFFECT"}}}]}
run data modify block ~ ~ ~ Items[{components:{"minecraft:potion_contents":{potion:"minecraft:MODIFIER_EFFECT"}}}]
merge value {components:{"minecraft:custom_name":"NAME (SIZE)"}}
""".replace(
    "\n", " "
).strip()

In [5]:
for effect, coffee, variants in coffees:
    print()
    base = template.replace("EFFECT", effect).replace("NAME", coffee)
    print(base.replace("MODIFIER_", "").replace("SIZE", "12oz"))
    if "long" in variants:
        print(base.replace("MODIFIER_", "long_").replace("SIZE", "16oz"))
    if "strong" in variants:
        print(base.replace("MODIFIER_", "strong_").replace("SIZE", "Espresso"))


execute if block ~ ~ ~ minecraft:brewing_stand{Items:[{components:{"minecraft:potion_contents":{potion:"minecraft:night_vision"}}}]} run data modify block ~ ~ ~ Items[{components:{"minecraft:potion_contents":{potion:"minecraft:night_vision"}}}] merge value {components:{"minecraft:custom_name":"Auromar Geisha (12oz)"}}
execute if block ~ ~ ~ minecraft:brewing_stand{Items:[{components:{"minecraft:potion_contents":{potion:"minecraft:long_night_vision"}}}]} run data modify block ~ ~ ~ Items[{components:{"minecraft:potion_contents":{potion:"minecraft:long_night_vision"}}}] merge value {components:{"minecraft:custom_name":"Auromar Geisha (16oz)"}}

execute if block ~ ~ ~ minecraft:brewing_stand{Items:[{components:{"minecraft:potion_contents":{potion:"minecraft:leaping"}}}]} run data modify block ~ ~ ~ Items[{components:{"minecraft:potion_contents":{potion:"minecraft:leaping"}}}] merge value {components:{"minecraft:custom_name":"Legender Especial (12oz)"}}
execute if block ~ ~ ~ minecraft:br

## Everything Else

...is honestly pretty straightforward and easy enough to copy/paste. We'll see.